In [2]:
import MDAnalysis as mda
import numpy as np
from numpy import linalg 
import pandas as pd
import matplotlib.pyplot as plt

from MLDynamics import io, properties

####################
# Define constants #
####################

PDB_COORDINATE_COLUMNS = [5,6,7]

# Project Directory
# Where are all the files located
WORKING_DIR = "/scratch/masauer2/FMA_KEMP/KE_ML/classic_study/template_R1_0"

# Subdirectory that the trajectories (.trr) are located in
# i.e each replica will be in file 02-sample_X (where X ranges from 0 to 19)
TRAJECTORY_DIR = "08-converged/02-sample_"
TRAJECTORY_NAME = "sample-NPT_pbc_rot+trans.trr"

# Number of replicas
N_REPLICAS = 1

# Reference Structure - this is only used to import the trr file w/ MDAnalysis
# We redefine the reference structure to be the first timestep of each trajectory
# See variable array - startTime
GRO_FILEPATH = "/scratch/masauer2/FMA_KEMP/KE_ML/classic_study/template_R1_0/00-prep/prot.gro"

def trunc(values, decs=0):
    return np.trunc(values*10**decs)/(10**decs)

## Step 1 - Use MD Analysis to read in trajectory

Please ensure that you have an enviornment setup that includes the following packages
- MDAnalysis
- numpy
- pandas
- matplotlib

All simulations were performed using the GROMACS 2022.5 software package. The AMBER99SB-ILDN force fied was used to describe the potential energy of the protein. Salt was added at a concentration of 150 nM. 

Energy minimization was performed using the steepest descent algorithmf or 1000 steps. Then, the system was equilibrated in the isobaric–isothermal (NPT) ensemble at 300 K and 1 bar for 100 ps with a 1 fs timestep, a velocity rescaling thermostat5 with a
1.0 ps time constant, and a stochastic cell rescaling barostat with a time constant of 2.0 ps.

Then, we performed two separate studies: a classic MD and an enhanced MD study. Each study follows the same protocol steps.

In the classic MD study, we perform a 100 ns simulation in the NPT ensemble to sample a configuration every 5 ns (for a total of 20 configurations). Each configuration becomes the starting point for an independent 100 ns NPT simulation - for a total of 2000 ns of simulation. We constrain all bonds involving hydrogens and use a 2 fs timestep. We also employ a Nose Hoover thermostate with a 1 ps timestep and a Parrinello-Rahman barostate with a 2.0 ps time constant. Short-ranged electrostatic and Lennard-Jones
interactions were treated with a 10 Å real–space cutoff with energy and pressure corrections
for dispersion interactions. Long–ranged electrostatic interactions were treated with the
Particle Mesh Ewald algorithm, using a 1.2 Å grid. 

In the enhanced MD study, we run 20 replicas of well-tempered metadynamics along FRESEAN Modes 7 and 8 at 0 THz at 300K. In WT-metadynamics, external bias potential is deposited at a fixed rate (termed Gaussians) - allowing for the exploration of high free energy regions along the selected collective variables. 
Gaussian functions were deposited at the current position in CV space every 1.0 ps with an initial height of 1.0 kJ/mol and a an initial width of 0.001 kJ/mol. The unitless bias factor was set to 10 for all cases.

In [3]:
# trajectoryNames stores the name of each trajectory 
# then we will import all replicas into one long trajectory

# Read in trajectories - already unwrapped and centered
trajectoryNames = []
for traj in range(N_REPLICAS):
    trr = f"{WORKING_DIR}/{TRAJECTORY_DIR}{traj}/{TRAJECTORY_NAME}"
    trajectoryNames.append(trr)

# system = all 20 replicas
system = mda.Universe(GRO_FILEPATH,trajectoryNames)
nTimes = len(system.trajectory)
nAtoms = len(system.atoms)

# Now read in the coordinates from the system
coordinates = np.zeros((nTimes, nAtoms, 3))
for ts in system.trajectory:
    coordinates[ts.frame] = ts.positions
coordinates = coordinates.reshape((nTimes, nAtoms * 3))

# Extra information that will be used to construct final datastructure (df)
atom_names = system.atoms.names
res_names = system.atoms.resnums
atom_masses = system.atoms.masses

# An unique identifier for all features
column_names = [f"{res_names[iter]}_{atom_names[iter]}" for iter in range(len(atom_names))]

# The mass of each DEGREE OF FREEDOM (not atom!!!)
DOF_masses = [atom_masses[atom] for atom in range(len(atom_masses)) for DOF in ["X", "Y", "Z"]]

## Step 2 - Compute the displacement along the trajectory 

Displacement is just the difference between the initial distance vector and current time distance vector. Since we have multiple replicas, we must denote the start time associated with each frame i.e for the first 5000 frames, the initial distance vector is frame 0. For frames 5001 - 10002, the initial distance vector is frame 5001.

In [4]:
displacement = np.zeros((np.shape(coordinates)[0], np.shape(coordinates)[1]))

# Reference time i.e starting time for each simulations based on total time
nTimesPerSimulation = nTimes/N_REPLICAS
startTime = np.zeros(nTimes, dtype=int)
simulationIdx = np.zeros(nTimes, dtype=int)
simulationTime = np.zeros(nTimes, dtype=int)
for dt in range(nTimes):
    startTime[dt] = int(dt/nTimesPerSimulation) * nTimesPerSimulation
    simulationIdx[dt] = int(dt/nTimesPerSimulation)
    simulationTime[dt] = dt % nTimesPerSimulation

# Compute displacement
for timestep in range(len(coordinates)):
    displacement[timestep] = coordinates[timestep] - coordinates[startTime[timestep]]

## Step 3 - Compute the displacement projection along FRESEAN Modes 7 and 8 at 0 THz

The displacement projection ($proj_{D,M}$) is computed for each timestep of the trajectory, $D_t$, on each FRESEAN mode, $M$.

$$
proj_{D,M}(t) = \textbf{M} \cdot \textbf{D} = \textbf{M} \cdot (\textbf{D}_{t} - \textbf{D}_{ref})
$$

The reference point, $\textbf{D}_{ref}$, is defined as $\textbf{D}_{t=0}$ for each replica i.e we compute the displacement w.r.t the first timestep and project it on each FRESEAN Mode. Units are in Angstroms (MDAnalysis units). 

The function for computing the projection can be found at the `MLDynamics.properties` package.

In [5]:
# FRESEAN Mode file that is used as the collective variable file for plumed
mode = "/scratch/masauer2/FMA_KEMP/KE_ML/metad_study/template_R1_metad/05-ModeProj/plumed-mode-input.pdb"

# We have one reference structure
nStructure = 1

# We have two modes -> 0 THz Mode 7 and 0 THz Mode 8
nModes = 2

eigenvector = np.array(pd.read_csv(mode, skiprows = lambda x: x in [(nAtoms+2)*i - 1 for i in range(1,1+nModes)] or x in [(nAtoms+2)*i for i in range(1,1+nModes)] or x in [0,(nAtoms+2)*(nModes+nStructure)-1], header=None, delim_whitespace=True)).reshape((nModes+nStructure,nAtoms,10))

# We will ignore the reference structure and read in the two modes
mode_coordinates = eigenvector[nStructure:,:,PDB_COORDINATE_COLUMNS].reshape((2,nAtoms*3))

# Normalize!!
mode_coordinates_norm = [mode_coordinates[i]/(np.linalg.norm(mode_coordinates[i])) for i in range(nModes)]

/tmp/ipykernel_1624080/562207430.py:10: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  eigenvector = np.array(pd.read_csv(mode, skiprows = lambda x: x in [(nAtoms+2)*i - 1 for i in range(1,1+nModes)] or x in [(nAtoms+2)*i for i in range(1,1+nModes)] or x in [0,(nAtoms+2)*(nModes+nStructure)-1], header=None, delim_whitespace=True)).reshape((nModes+nStructure,nAtoms,10))


In [6]:
# Extra information that will be used to construct final datastructure (df)
mode_resnum = eigenvector[nStructure:,:,4].reshape((2,nAtoms))[0]
mode_resname = eigenvector[nStructure:,:,2].reshape((2,nAtoms))[0]
mode_column_names = [f"{mode_resnum[iter]}_{mode_resname[iter]}_{DOF}" for iter in range(len(mode_resnum)) for DOF in ["X", "Y", "Z"]]

In [7]:
# Reshape so the 3 DOF per atom are in 1 line
mode_coordinates_reshaped = [mode_coordinates_norm[i].reshape((nAtoms,3)) for i in range(nModes)]

# Number of frames = Number of times (this variable can be reset to a small value for testing)
nFrames = 10

# Reshape displacement matrix so that all DOF per atom are one vector 
displacement_perAtom = displacement[:nFrames].reshape((nFrames,nAtoms,3))

# Array to store displacement
projection_perAtom = np.zeros((nModes, np.shape(displacement_perAtom)[0],nAtoms))

for i in range(np.shape(displacement_perAtom)[0]):
    for j in range(np.shape(displacement_perAtom)[1]):
        # Hard coded for 2 modes - extend for any number of modes
        for modeNum in range(nModes):
            projection_perAtom[modeNum,i,j] = properties.projection(displacement_perAtom[i,j],mode_coordinates_reshaped[modeNum][j],nDim=3)
    

## Step 4 - Output the displacement projection to a Data Frame

For our purposes, ML_INPUT will be the starting structure of our machine learning pipeline.

Notes about the dataset:

- This code (run_replicas.ipynb) generates ML_INPUT for EACH protein. This will need to be regenerated for both unevolved and evolved proteins.
- Labels are added after the fact using helper functions from `MLDynamics.io` package.

Notes about the features: 

- Each element of the dataframe is a tuple/list containing both the mode 7 and mode 8 projection. Helper functions in `MLDynamics.io` package can be used to parse this data structure

Notes for Chris:

- We might want to create a `Trajectory` Class that specifies a format for the data frame (columns = atoms, rows = timesteps, elements = lists).
- Additionally, we might want to put the helper methods in that class.

In [8]:
displacementProjection_Modes = np.zeros((np.shape(displacement_perAtom)[0],nAtoms), dtype=object)
for row in range(len(displacementProjection_Modes)):
    for col in range(len(displacementProjection_Modes[row])):
        displacementProjection_Modes[row, col] = [projection_perAtom[0, row, col], projection_perAtom[1, row, col]]

ML_INPUT = pd.DataFrame(displacementProjection_Modes, columns=column_names)

ML_INPUT_WITHLABELS = io.add_labels(ML_INPUT, ["Simulation Number", "Simulation Time", "Identity"], [simulationIdx, simulationTime, ["Unevolved"]*np.shape(ML_INPUT)[0]])
ML_INPUT_WITHLABELS.to_pickle("ML_INPUT_test.pkl")  

In [12]:
def trunc(values, decs=0):
    return np.trunc(values*10**decs)/(10**decs)

trunc(displacementProjection_Modes, decs=2)

TypeError: type list doesn't define __trunc__ method